# Create a dual-axis chart comparing the average installs and revenue for free vs. paid apps within the top 3 app categories. Apply filters to exclude apps with fewer than 10,000 installs and revenue below $10,000 and android version should be more than 4.0 as well as size should be more than 15M and content rating should be Everyone and app name should not have more than 30 characters including space and special character .this graph should work only between 1 PM IST to 2 PM IST apart from that time we should not show this graph in dashboard itself.



In [15]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime,time
import numpy as np
import regex as re

# load dataset

In [16]:
df=pd.read_csv("google playstore data.csv")
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


# datacleaning and processing 

In [ ]:
df=df[df["Installs"].str.contains(r'\d',regex=True,na=False)] #remove non-numerical values 
df['Installs']=df['Installs'].str.replace('[+,]','',regex=True).astype(float)

df['Price']=df['Price'].str.replace('[$]','',regex=True).astype(float)

def clean_size(size):
    if isinstance(size, str):
        if 'M' in size:
            return float(size.replace('M', ''))
        elif 'k' in size:
            return float(size.replace('k', '')) / 1000  # convert kB to MB
    return np.nan
df['Size'] = df['Size'].apply(clean_size)
# print(df['Size'].describe())
df['Revenue'] = np.where(df['Type'] == 'Paid', df['Installs'] * df['Price'], 0)
# # print(df['Revenue'].describe())
def clean_android_ver(version):
    match = re.search(r'\d+(\.\d+)?', str(version))
    return float(match.group()) if match else np.nan
df['Android Ver'] = df['Android Ver'].apply(clean_android_ver)
# # print(df['Android Ver'].describe())

# filltering data

In [18]:

filtered_df = df[
    (df['Installs'] >= 10000) &
    (df['Revenue'] >= 10000) &
    (df['Android Ver'] > 4.0) &
    (df['Size'] > 15) &
    (df['Content Rating'] == 'Everyone') &
    (df['App'].str.len() <= 30)
]
filtered_df.head()


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Revenue
853,Toca Life: City,EDUCATION,4.7,31085,24.0,500000.0,Paid,3.99,Everyone,Education;Pretend Play,"July 6, 2018",1.5-play,4.4,1995000.0
854,Toca Life: Hospital,EDUCATION,4.7,3528,24.0,100000.0,Paid,3.99,Everyone,Education;Pretend Play,"June 12, 2018",1.1.1-play,4.4,399000.0
1335,Meditation Studio,HEALTH_AND_FITNESS,4.6,1026,29.0,10000.0,Paid,3.99,Everyone,Health & Fitness,"May 15, 2018",1.0.6,4.3,39900.0
1831,The Game of Life,GAME,4.4,18621,63.0,100000.0,Paid,2.99,Everyone,Board,"July 4, 2018",2.1.2,4.4,299000.0
1833,The Room: Old Sins,GAME,4.9,21119,48.0,100000.0,Paid,4.99,Everyone,Puzzle,"April 18, 2018",1.0.1,4.4,499000.0


In [19]:
grouped = filtered_df.groupby(['Category', 'Type']).agg({
    'Installs': 'mean',
    'Revenue': 'mean'
}).reset_index()
grouped.head()

,Category,Type,Installs,Revenue
0,EDUCATION,Paid,300000.000000,1197000.0
1,FAMILY,Paid,381428.571429,796900.0
2,GAME,Paid,71250.000000,279287.5
3,HEALTH_AND_FITNESS,Paid,10000.000000,54900.0
4,LIFESTYLE,Paid,10000.000000,3999900.0


In [20]:
top_3_cats=filtered_df['Category'].value_counts().head(3).index
print(top_3_cats)
top_3_groups=grouped[grouped['Category'].isin(top_3_cats)]
top_3_groups

Index(['GAME', 'FAMILY', 'PHOTOGRAPHY'], dtype='object', name='Category')


,Category,Type,Installs,Revenue
1,FAMILY,Paid,381428.571429,796900.0
2,GAME,Paid,71250.000000,279287.5
8,PHOTOGRAPHY,Paid,1000000.000000,5990000.0


In [21]:

current_time = datetime.now().time()
start_time = time(13, 0)  
end_time = time(14, 0)    


if start_time <= current_time <= end_time:
    fig = go.Figure()

 
    fig.add_trace(go.Bar(
        x=top_3_groups['Category'],
        y=top_3_groups['Installs'],
        name='Average Installs',
        marker_color='blue'
    ))


    fig.add_trace(go.Scatter(
        x=top_3_groups['Category'],
        y=top_3_groups['Revenue'],
        mode='lines+markers',
        name='Average Revenue',
        marker_color='red',
        yaxis='y2'
    ))

    
    fig.update_layout(
        title='Comparison of Average Installs and Revenue for Top 3 App Categories (Paid Apps)',
        xaxis_title='App Categories',
        yaxis=dict(title='Average Installs'),
        yaxis2=dict(
            title='Average Revenue ($)',
            overlaying='y',
            side='right'
        ),
        legend=dict(x=0.1, y=1.1),
        height=600,
        width=1000
    )

    fig.show()
else:
    print("The chart can only be viewed between 1 PM and 2 PM IST.")

The chart can only be viewed between 1 PM and 2 PM IST.
